## MLflow's Model Registry

In [ ]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [6]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

#client.

In [11]:
client.create_experiment(name="my-cool-experiment")

'7'

Let's check the latest versions for the experiment with id `1`...

In [20]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='6',
    filter_string="metrics.accuracy > 0.7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.accuracy ASC"]
)

In [21]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['accuracy']:.4f}")

run id: 9ae9e827dc7844a082a60551fba7d3b9, rmse: 0.9469
run id: 80db568af11b4f9aafb461a150608565, rmse: 0.9916
run id: 0cb2d1d3120d40ed99a56187f14351cb, rmse: 0.9919


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [27]:
run_id = "80db568af11b4f9aafb461a150608565"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="from_vscode_mnist")

Registered model 'from_vscode_mnist' already exists. Creating a new version of this model...
2025/07/14 16:31:21 WARNING mlflow.tracking._model_registry.fluent: Run with id 80db568af11b4f9aafb461a150608565 has no artifacts at artifact path 'model', registering model based on models:/m-13b91b67c0fd476f865617e8b32c091a instead


Created version '3' of model 'from_vscode_mnist'.


<ModelVersion: aliases=[], creation_timestamp=1752490881412, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1752490881412, metrics=None, model_id=None, name='from_vscode_mnist', params=None, run_id='80db568af11b4f9aafb461a150608565', run_link=None, source='models:/m-13b91b67c0fd476f865617e8b32c091a', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [31]:
model_name = "from_vscode_mnist"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 2, stage: Staging
version: 3, stage: None


C:\Users\andre\AppData\Local\Temp\ipykernel_37660\1281974936.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [30]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

C:\Users\andre\AppData\Local\Temp\ipykernel_37660\1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1752490631101, current_stage='Staging', deployment_job_state=None, description=None, last_updated_timestamp=1752490911553, metrics=None, model_id=None, name='from_vscode_mnist', params=None, run_id='80db568af11b4f9aafb461a150608565', run_link=None, source='models:/m-13b91b67c0fd476f865617e8b32c091a', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [33]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=3,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1752490881412, current_stage='None', deployment_job_state=None, description='The model version 1 was transitioned to Staging on 2025-07-14', last_updated_timestamp=1752491114770, metrics=None, model_id=None, name='from_vscode_mnist', params=None, run_id='80db568af11b4f9aafb461a150608565', run_link=None, source='models:/m-13b91b67c0fd476f865617e8b32c091a', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_csv(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)




In [36]:
df = read_dataframe("data/green_tripdata_2021-03.csv")

/var/folders/42/f9s_rgk15078ym2w50_xtc180000gq/T/ipykernel_5486/3050441246.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [37]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/Users/cristian.martinez/Repositories/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [38]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [39]:
X_test = preprocess(df, dv)

In [40]:
target = "duration"
y_test = df[target].values

# My code


In [34]:
import mlflow
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_test = X_test.reshape(-1, 28*28) / 255.0

In [36]:
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    acc = accuracy_score(y_test, y_pred_classes)
    return {"accuracy": acc}

In [37]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
CPU times: total: 3.2 s
Wall time: 10.5 s


{'accuracy': 0.9756}

In [38]:
client.transition_model_version_stage(
    name=model_name,
    version=2,
    stage="Production",
    archive_existing_versions=True
)

C:\Users\andre\AppData\Local\Temp\ipykernel_37660\1703833061.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1752490808428, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1752491533025, metrics=None, model_id=None, name='from_vscode_mnist', params=None, run_id='80db568af11b4f9aafb461a150608565', run_link=None, source='models:/m-13b91b67c0fd476f865617e8b32c091a', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [39]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
CPU times: total: 984 ms
Wall time: 803 ms


{'accuracy': 0.9756}